In [1]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import requests

In [2]:
apikey = 'jQifsxeX6qqydRZ7BtPnKhoCc5mIrwepALZf9ZfunvpNRkvh78MSZcIw1W3DEfLX'

In [3]:
client = Client(apikey)

In [71]:
def get_market_cap(quote="USDT"):
    
    resp = requests.get("https://www.binance.com/bapi/asset/v2/public/asset-service/product/get-products")
    
    market_cap=pd.DataFrame(resp.json()['data'])
    market_cap=market_cap[market_cap['q']==quote]
    
    market_cap=market_cap[['an','qn','s','b','q','c','cs']]
    market_cap['c']=market_cap['c'].astype(float)
    
    market_cap.columns=['Long name','Quote Name','Ticker','Short Name','Quote Short Name','Close','Supply']
    market_cap['Market Cap']=market_cap['Close']*market_cap['Supply']
    
    market_cap=market_cap.sort_values(by='Market Cap',ascending=False)
    

    return market_cap

In [4]:

def get_price(ticker_list,date="01/01/2022"):
    

    price=pd.DataFrame()

    for ticker in ticker_list:

        
        numeric_columns =  ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                        'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
        
        data = pd.DataFrame(client.get_historical_klines(ticker, Client.KLINE_INTERVAL_1DAY, date),columns=numeric_columns)
        data['Close Time']=pd.to_datetime(data['Close Time'], unit='ms').dt.date
        data=data.set_index('Close Time')

        price[ticker]=data['Close'].astype(float)
        
        #price=pd.concat([price,temp[ticker]],axis=1)
        price=pd.concat([price,price[ticker]],axis=1)
        
    return price